# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [19]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_combined = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_combined

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

In [3]:
# Combine the data into a single dataset
school_data_combined = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_combined

# Calculate the Totals (Schools and Students)
total_number_schools = len(school_data_combined["School ID"].unique())
total_number_schools

total_number_students = len(school_data_combined["Student ID"].unique())
total_number_students

# Calculate the Total Budget
total_budget = school_data["budget"].sum()
total_budget

# Calculate the Average Scores
average_math_score = student_data["math_score"].mean()
average_math_score

average_reading_score = student_data["reading_score"].mean()
average_reading_score

student_data["#passing_math"] = student_data["math_score"] >= 70
percent_passing_math = ((student_data["#passing_math"]).mean())*100
percent_passing_math

student_data["#passing_reading"] = student_data["reading_score"] >= 70
percent_passing_reading = ((student_data["#passing_reading"]).mean())*100
percent_passing_reading

# Calculate the Percentage Pass Rates
overall_average_score = (average_math_score + average_reading_score)/2
overall_average_score

# Display the data frame
district_results = [{"Total Schools": total_number_schools, 
            "Total Students": total_number_students, 
            "Total Budget": total_budget, 
            "Average Math Score":  round(average_math_score,2), 
            "Average Reading Score":  round(average_reading_score,2), 
           "% Passing Math": round(percent_passing_math,2),
           "% Passing Reading": round(percent_passing_reading,2),
            "% Overall Passing Rate": round(overall_average_score,2)}]
district_summary_table = pd.DataFrame(district_results)

# Formatting
district_summary_table["% Passing Math"] = district_summary_table["% Passing Math"].map("{:,.2f}%".format)
district_summary_table["% Passing Reading"] = district_summary_table["% Passing Reading"].map("{:,.2f}%".format)
district_summary_table["% Overall Passing Rate"] = district_summary_table["% Overall Passing Rate"].map("{:,.2f}%".format)
district_summary_table["Total Budget"] = district_summary_table["Total Budget"].map("${:,.2f}".format)
district_summary_table["Total Students"] = district_summary_table["Total Students"].map("{:,}".format)

# Display
district_summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.43%


## School Summary

In [31]:
# Determine the School Type
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count
students_per_school = school_data_combined["school_name"].value_counts()

# Calculate the total school budget and per capita spending
school_budget = school_data_combined.groupby(["school_name"]).mean()["budget"]
student_budget = school_budget / students_per_school 

# Calculate the average test scores
average_math = school_data_combined.groupby(["school_name"]).mean()["math_score"]
average_reading = school_data_combined.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered data frame
passing_math = school_data_combined[(school_data_combined["math_score"] >= 70)]
passing_reading = school_data_combined[(school_data_combined["reading_score"] >= 70)]

passing_math = passing_math.groupby(["school_name"]).count()["student_name"] / students_per_school * 100
passing_reading = passing_reading.groupby(["school_name"]).count()["student_name"] / students_per_school * 100
overall = (passing_math + passing_reading) / 2

# Convert to data frame
school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": students_per_school,
    "Total School Budget": school_budget,
    "Per Student Budget": student_budget,
    "Average Math Score": average_math,
    "Average Reading Score": average_reading,
    "% Passing Math": passing_math,
    "% Passing Reading": passing_reading,
    "% Overall Passing Rate": overall})

# Minor data munging
school_summary = school_summary[["School Type", 
                                "Total Students", 
                                "Total School Budget", 
                                "Per Student Budget",
                                "Average Math Score",
                                "Average Reading Score", 
                                "% Passing Math", 
                                "% Passing Reading", 
                                "% Overall Passing Rate"]]

school_summary["Total School Budget"] = school_summary["Total Students"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$4,976.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,858.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$2,949.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$2,739.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$1,468.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$4,635.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,$427.00,$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$2,917.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$4,761.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,$962.00,$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

In [32]:
# Sort and show top five schools
top_schools = school_summary.sort_values(["% Overall Passing Rate"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,858.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,635.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,$962.00,$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$1,468.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$2,283.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

In [33]:
# Sort and show bottom five schools
bottom_schools = school_summary.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$3,999.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$2,949.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$2,917.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$4,761.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$2,739.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

In [13]:
# Create data series of scores by grade levels using conditionals & Group each by school name
nineth_grade= school_data_combined[school_data_combined["grade"] == "9th"].groupby("school_name").mean()["math_score"]
tenth_grade = school_data_combined[school_data_combined["grade"] == "10th"].groupby("school_name").mean()["math_score"]
eleventh_grade = school_data_combined[school_data_combined["grade"] == "11th"].groupby("school_name").mean()["math_score"]
twelveth_grade= school_data_combined[school_data_combined["grade"] == "12th"].groupby("school_name").mean()["math_score"]

# Combine series into single data frame
math_grade_dataframe = pd.DataFrame({"Ninth Grade":nineth_grade, "Tenth Grade":tenth_grade, 
                                     "Eleventh Grade":eleventh_grade, "Twelveth Grade":twelveth_grade})

# Minor data munging
math_grade_dataframe[["Ninth Grade","Tenth Grade","Eleventh Grade","Twelveth Grade"]] = math_grade_dataframe[["Ninth Grade","Tenth Grade","Eleventh Grade","Twelveth Grade"]].applymap("{:.2f}".format)

# Display the data frame
math_grade_dataframe

,Ninth Grade,Tenth Grade,Eleventh Grade,Twelveth Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [14]:
# Create data series of scores by grade levels using conditionals & Group each by school name
nineth_grade=  school_data_combined[school_data_combined["grade"] == "9th"].groupby("school_name").mean()["reading_score"]
tenth_grade =  school_data_combined[school_data_combined["grade"] == "10th"].groupby("school_name").mean()["reading_score"]
eleventh_grade =  school_data_combined[school_data_combined["grade"] == "11th"].groupby("school_name").mean()["reading_score"]
twelveth_grade=  school_data_combined[school_data_combined["grade"] == "12th"].groupby("school_name").mean()["reading_score"]

# Combine series into single data frame
reading_grade_dataframe = pd.DataFrame({"Ninth Grade":nineth_grade, "Tenth Grade":tenth_grade, 
                                     "Eleventh Grade":eleventh_grade, "Twelveth Grade":twelveth_grade})
# Minor data munging
reading_grade_dataframe[["Ninth Grade","Tenth Grade","Eleventh Grade","Twelveth Grade"]] = reading_grade_dataframe[["Ninth Grade","Tenth Grade","Eleventh Grade","Twelveth Grade"]].applymap("{:.2f}".format)

# Display the data frame
reading_grade_dataframe

,Ninth Grade,Tenth Grade,Eleventh Grade,Twelveth Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [41]:
# Establish the bins -- choose any set of bins you would like, but see below for testing bins
# to test, set your bins as follows: [0, 585, 615, 645, 675]
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Categorize the spending based on the bins

school_summary["Spending Ranges (Per Student)"] = pd.cut(student_budget, spending_bins, labels=group_names)
average_math = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
average_reading = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
passing_math = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
passing_reading = school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall = (spending_math_scores + spending_reading_scores) / 2


# Assemble into data frame
spending_summary = pd.DataFrame({"Average Math Score" : average_math,
                                 "Average Reading Score": average_reading,
                                 "% Passing Math": passing_math,
                                 "% Passing Reading":passing_reading,
                                 "% Overall Passing Rate": overall})

# Minor data munging
spending_summary = spending_summary[["Average Math Score", 
                                     "Average Reading Score", 
                                     "% Passing Math", "% Passing Reading",
                                     "% Overall Passing Rate"]]

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,83.694607
$585-615,83.599686,83.885211,94.230858,95.900287,83.742449
$615-645,79.079225,81.891436,75.668212,86.106569,80.485330
$645-675,76.997210,81.027843,66.164813,81.133951,79.012526


## Scores by School Size

In [42]:
# Establish the bins 
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize the spending based on the bins
school_summary["School Size"] = pd.cut(school_summary["Total Students"], size_bins, labels=group_names)

# Calculate the scores based on bins
average_math = school_summary.groupby(["School Size"]).mean()["Average Math Score"]
average_reading = school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
passing_math = school_summary.groupby(["School Size"]).mean()["% Passing Math"]
passing_reading = school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
overall = (passing_math + passing_reading) / 2

# Assemble into data frame
size_summary = pd.DataFrame({"Average Math Score" : average_math,
                             "Average Reading Score": average_reading,
                             "% Passing Math": passing_math,
                             "% Passing Reading": passing_reading,
                             "% Overall Passing Rate": overall})

# Minor data munging
size_summary = size_summary[["Average Math Score", 
                             "Average Reading Score", 
                             "% Passing Math", "% Passing Reading",
                             "% Overall Passing Rate"]]
# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

In [44]:
# Type | Average Math Score | Average Reading Score | % Passing Math | % Passing Reading | % Overall Passing Rate
average_math = school_summary.groupby(["School Type"]).mean()["Average Math Score"]
average_reading = school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
passing_math = school_summary.groupby(["School Type"]).mean()["% Passing Math"]
passing_reading = school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
overall = (passing_math + passing_reading) / 2

# Assemble into data frame
type_summary = pd.DataFrame({"Average Math Score" : average_math,
                             "Average Reading Score": average_reading,
                             "% Passing Math": passing_math,
                             "% Passing Reading": passing_reading,
                             "% Overall Passing Rate": overall})

# Minor data munging
type_summary = type_summary[["Average Math Score", 
                             "Average Reading Score",
                             "% Passing Math",
                             "% Passing Reading",
                             "% Overall Passing Rate"]]

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
